First, create the model. This must match the model used in the interactive training notebook.

In [2]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!git clone https://github.com/NVIDIA-AI-IOT/jetracer

Cloning into 'jetracer'...
remote: Enumerating objects: 1105, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 1105 (delta 117), reused 114 (delta 114), pack-reused 949
Receiving objects: 100% (1105/1105), 7.63 MiB | 6.18 MiB/s, done.
Resolving deltas: 100% (609/609), done.


In [4]:
#!rm -rf /content/data_obstacle_detection
#!rm -rf /content/data_perfect_route
#!rm -rf /content/data_self_correct

In [5]:
!cp /content/drive/MyDrive/road_following_A.zip /content/jetracer/notebooks/ -r
!cp /content/drive/MyDrive/road_following_B.zip /content/jetracer/notebooks/ -r
!cp /content/drive/MyDrive/road_following_D.zip /content/jetracer/notebooks/ -r
!cp /content/drive/MyDrive/road_following_T.zip /content/jetracer/notebooks/ -r

In [6]:
!unzip /content/jetracer/notebooks/road_following_B.zip
!unzip /content/jetracer/notebooks/road_following_A.zip
!unzip /content/jetracer/notebooks/road_following_D.zip
!unzip /content/jetracer/notebooks/road_following_T.zip

Archive:  /content/jetracer/notebooks/road_following_B.zip
   creating: road_following_B/apex/
  inflating: road_following_B/apex/100_108_5c505c28-825a-11eb-9f82-24ee9a2edc3b.jpg  
  inflating: road_following_B/apex/101_108_4927e378-825a-11eb-9f82-24ee9a2edc3b.jpg  
  inflating: road_following_B/apex/101_95_0178f544-825a-11eb-9f82-24ee9a2edc3b.jpg  
  inflating: road_following_B/apex/102_107_5020d0a4-825a-11eb-9f82-24ee9a2edc3b.jpg  
  inflating: road_following_B/apex/102_109_476ed7d0-825a-11eb-9f82-24ee9a2edc3b.jpg  
  inflating: road_following_B/apex/104_108_1bbf0984-825a-11eb-9f82-24ee9a2edc3b.jpg  
  inflating: road_following_B/apex/104_98_facfed24-8259-11eb-9f82-24ee9a2edc3b.jpg  
  inflating: road_following_B/apex/112_111_4c50a6e8-825a-11eb-9f82-24ee9a2edc3b.jpg  
  inflating: road_following_B/apex/114_103_3fc5946a-825a-11eb-9f82-24ee9a2edc3b.jpg  
  inflating: road_following_B/apex/117_109_42d8df7c-825a-11eb-9f82-24ee9a2edc3b.jpg  
  inflating: road_following_B/apex/117_109_532e

In [7]:
import torchvision.transforms as transforms
from jetracer.notebooks.xy_dataset import XYDataset

TASK = 'road_following'

CATEGORIES = ['apex']

DATASETS = ['A', 'B', 'D', 'T']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)

In [8]:
import cv2
import ipywidgets
import traitlets
from IPython.display import display

# from jetcam.utils import bgr8_to_jpeg
# from jupyter_clickable_image_widget import ClickableImageWidget


# initialize active dataset
dataset = datasets[DATASETS[0]]

# # unobserve all callbacks from camera in case we are running this cell for second time
# camera.unobserve_all()

# # create image preview
# camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
# snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
# traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')


# def save_snapshot(_, content, msg):
#     if content['event'] == 'click':
#         data = content['eventData']
#         x = data['offsetX']
#         y = data['offsetY']

#         # save to disk
#         dataset.save_entry(category_widget.value, camera.value, x, y)

#         # display saved snapshot
#         snapshot = camera.value.copy()
#         snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
#         snapshot_widget.value = bgr8_to_jpeg(snapshot)
#         count_widget.value = dataset.get_count(category_widget.value)

# camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([]),
    dataset_widget,
    category_widget,
    count_widget
])

display(data_collection_widget)

In [9]:
import torch
import torchvision

device = torch.device('cuda')
output_dim = 2 * len(dataset.categories)  # x, y coordinate for each category

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# DENSENET 121
# model = torchvision.models.densenet121(pretrained=True)
# model.classifier = torch.nn.Linear(model.num_features, output_dim)

model = model.to(device)

model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value='road_following_model.pth')

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)

def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])


display(model_widget)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 110MB/s]


In [10]:
import threading
import time
from jetracer.notebooks.utils import preprocess
import torch.nn.functional as F

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
#prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

def live(state_widget, model, camera, prediction_widget):
    global dataset
    while state_widget.value == 'live':
        image = camera.value
        preprocessed = preprocess(image)
        output = model(preprocessed).detach().cpu().numpy().flatten()
        category_index = dataset.categories.index(category_widget.value)
        x = output[2 * category_index]
        y = output[2 * category_index + 1]

        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))

        prediction = image.copy()
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        #prediction_widget.value = bgr8_to_jpeg(prediction)

def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([state_widget])

display(live_execution_widget)

In [11]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget

    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()

        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, category_idx, xy in iter(train_loader):
                # send data to device
                images = images.to(device)
                xy = xy.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute MSE loss over x, y coordinates for associated categories
                loss = 0.0
                for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                    loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
                loss /= len(category_idx)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                count = len(category_idx.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i

            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except Exception as e:
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    #state_widget.value = 'live'

train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))

train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

display(train_eval_widget)

In [12]:
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget, live_execution_widget]),
    train_eval_widget,
    model_widget
])

display(all_widget)